![Datadog Logo](images/dd_logo.png)

# Logging with Datadog Log Management

Welcome to Logging with Datadog Log Management workshop.

This repo contains a "dummy" Water System microservices project, a single page web application with microservices, already instrumented and analyzed using Datadog's APM and infrastructure products.

Using features like labels and pipelines, we'll see how log management can reduce your mean time to resolution should you encounter an issue with a given application.

The repo itself is run using Docker, along with `docker-compose`, which should now be bundled with [Windows](https://store.docker.com/editions/community/docker-ce-desktop-windows) and [MacOS](https://store.docker.com/editions/community/docker-ce-desktop-mac) versions of Docker. 

On Linux, Docker can be installed with your preferred method, and `docker-compose` should be available via a `pip install docker-compose`.

Using Docker allows us to set up multiple microservices locally, giving us an environment to rapidly build, test, and instrument our demo distributed system.

For the workshop, we'll be using some fresh, demo accounts. These don't have any other services submitting their Datadog account.

If you'd like to play around on your own, or on an account your company provides, and don't want to collide with other people's work, you'll want to pay attention to the `env` tag set in the workshop. 

You can set this to what you prefer, and have it be isolated in the Datadog drop down, for the purpose of this workshop it has been already set to `env:workshop` in your `docker-compose.yml` and application.

[Kirk Kaiser](http://www.twitter.com/burningion) has worked hard to make this workshop as helpful possible, but if you see something that could be improved, please feel free to create a [Github issue](https://github.com/burningion/distributed-tracing-with-apm-workshop) on the repo, or reach out via the [Datadog public slack](https://chat.datadoghq.com/). 

# Getting Started 

The first thing to do is to create a [Datadog account](https://www.datadoghq.com), and download this repository. 

Then run a `docker-compose up`, using the <DD_API_KEY> [from your trial account](https://app.datadoghq.com/account/settings#api).

Your command should look like the following on MacOS/Linux:

```bash
$ POSTGRES_USER=postgres POSTGRES_PASSWORD=123456 DD_API_KEY=<DD_API_KEY> docker-compose up --build
```

For Windows, the process of setting environment variables is a bit different:

```
PS C:\dev> $env:POSTGRES_USER=postgres
PS C:\dev> $env:POSTGRES_PASSWORD=123456
PS C:\dev> $env:DD_API_KEY=<DD_API_KEY>
PS C:\dev> docker-compose up --build
```

With the command run, you should see Docker start pulling down container images for the code. Afterwards, you'll be able to go to [http://localhost:5000/](http://localhost:5000/) and see the single page web app.

![Our Single Page App](images/dashboard.png)

Refresh the page, click around, add a pump, try adding a city. This begins to generate metrics, APM traces, and logs for your application

Tab back over to your console, and look over the container logs, go in your Datadog log management page and notice that there is no log yet... 

Our first mission should you choose to accept it, setup the Datadog Agent to start forwarding your logs into your Datadog Application.

# Gathering our first logs

![Empty Log explorer](images/logs_workshop/empty_log_explorer.png)

There is no log yet in your [Log Explorer page](https://app.datadoghq.com/logs), it's because the Datadog Agent is already configured to gather your APM traces and metrics but not your logs, in order to gather them you must add the following configuration lines in your `docker-compose.yml` file:

```
datadog:
  environment:
    (...)
    - DD_LOGS_ENABLED=true
    - DD_LOGS_CONFIG_CONTAINER_COLLECT_ALL=true
  volume:
    (...)
    - /opt/datadog-agent/run:/opt/datadog-agent/run:rw
```

| Configuration                                      | type         | Explanations                                    |
| :----                                              | :-----       | :-----                                          |
| `DD_LOGS_ENABLED=true`                             | env variable | Enable log collection                           |
| `DD_LOGS_CONFIG_CONTAINER_COLLECT_ALL=true`        | env variable | Enable log collection for all containers        |
| `/opt/datadog-agent/run:/opt/datadog-agent/run:rw` | volume       | Used to store pointers on container current log |

[Refer to the Datadog Agent log collection documentation to learn more](https://docs.datadoghq.com/logs/log_collection/docker/). 

Then restart your application:

1. Run `docker-compose stop && docker-compose rm`
2. Run `POSTGRES_USER=postgres POSTGRES_PASSWORD=123456 DD_API_KEY=<DD_API_KEY> docker-compose up --build`

----

*Note*: On some OS you might see this error popping:

```
ERROR: for agent  Cannot start service agent: b'Mounts denied: \r\nThe path /opt/datadog-agent/run\r\nis not shared ...
```

To fix it either give the mount permission to this folder on your machine, or remove `
/opt/datadog-agent/run:/opt/datadog-agent/run:rw` from the `docker-compose.yml` file.

------

Finaly, go in your Datadog application in [`Log -> Explorer`](https://app.datadoghq.com/logs/) and check your logs flowing.

![Log Flow](images/logs_workshop/log_flow.png)


# Using Label to correctly tag logs

As you can notice in the previous screenshot, all logs are currently showing up in the Datadog Log explorer view with the same service name `docker` -which is technically true- but in order to gain more visibility about which container emited which logs and in order to bind your logs with the previously implemented APM and metrics, let's use Labels to specify the `source` and the `service` tags for each container logs. 


* **The `source` tag is key to enable the integration pipeline**

    Datadog has a range of [Log supported integrations](https://docs.datadoghq.com/integrations/#cat-log-collection). In order to enable the [Log integrations pipeline](https://docs.datadoghq.com/logs/processing/pipelines/) in Datadog, pass the `source` name as a value for the source attribute with a docker label.

* **The `service` tag is key for binding metrics traces and logs.**

    The application is already instrumented for APM. Let's add the `service` tags to the `frontend`, `noder`, `pumps`, `redis`, `sensor`, `db` and `adminer` containers in order to be able to bind their traces and their logs together.

Update your docker-compose.yml with the following labels:

```
version: '3'
services:
  agent:
    (...)
    labels:
      com.datadoghq.ad.logs: '[{"source": "docker", "service": "agent"}]'

  frontend:
    (...)
    labels:
      com.datadoghq.ad.logs: '[{"source": "iot-frontend", "service": "iot-frontend"}}]'

  noder:
    (...)
    labels:
      com.datadoghq.ad.logs: '[{"source": "redis", "service": "noder"}]'
      
  pumps:
    (...)
    labels:
      com.datadoghq.ad.logs: '[{"source": "pumps-service", "service": "pumps-service"}]'

  redis:
    (...)
    labels:
      com.datadoghq.ad.logs: '[{"source": "redis", "service": "redis"}]'
      
  sensors:
    (...)
    labels:
      com.datadoghq.ad.logs: '[{"source": "nginx", "service": "sensors-api"}]'

  db:
    (...)
    labels:
      com.datadoghq.ad.logs: '[{"source": "postgres", "service": "postgres"}]'

  adminer:
    (...)
    labels:
      com.datadoghq.ad.logs: '[{"source": "adminer", "service": "adminer"}]'
```


Then restart your application:

1. Run `docker-compose stop && docker-compose rm`
2. Run `POSTGRES_USER=postgres POSTGRES_PASSWORD=123456 DD_API_KEY=<DD_API_KEY> docker-compose up --build`

And go in your Log explorer view to see the new `service` tags flowing in:

![Log flow with service](images/logs_workshop/log_flow_with_service.png)


This now allows us to switch between our log explorer view and the corresponding APM service:

![sensors_api_switch](images/logs_workshop/sensors_api_switch.png)

1. Open a log from `sensors-api` by clicking on it.
2. On top of the contextual panel click on the `sensors-api` Service name.
3. You should arrive on this page in Datadog APM:

![sensors-api service page](images/logs_workshop/sensors_api_service_page.png)


# Logging without limit

Now that our logs are correctly labeled we are able to manipulate them during their processing in Datadog.

Let's go in the [Pipeline page](https://app.datadoghq.com/logs/pipelines) of Datadog and see what we have:

The `source` tag already enabled the `Docker` and `Redis` integration pipeline:


![Pipeline_page](images/logs_workshop/pipeline_page.png)


#### Removing Agent log

In order to clean our log explorer from logs that are not relevant for our usecase let's implement an [index filter](https://docs.datadoghq.com/logs/logging_without_limits/#exclusion-filters). :

![index_filter_agent_log](images/logs_workshop/index_filter_agent_log.png)

Learn more about [Logging without limits](https://docs.datadoghq.com/logs/logging_without_limits/).


#### Removing Debug log

As a general best practice we also advise you to add an index filter on your Debug logs:

![removing_debug_logs](images/logs_workshop/removing_debug_logs.png)

Let's restart our application and see the impact of the previously set filters:

1. Run `docker-compose stop && docker-compose rm`
2. Run `POSTGRES_USER=postgres POSTGRES_PASSWORD=123456 DD_API_KEY=<DD_API_KEY> docker-compose up --build`

Our log explorer view now only contains logs from our containers and no more from the Datadog Agent all logs matching the following query: `service:agent` are no longer reporting: 

![agent_filtered_out](images/logs_workshop/agent_filtered_out.png)

# Live tail 

Now that we filtered out our Agent logs and all our Debug logs, our explorer view is cleaner but we might still want to consult those logs. 

It's still possible with the [Live tail page](https://app.datadoghq.com/logs/livetail) of Datadog.

The live tail page displays all logs after the Pipeline section but before the index filter one. If you enter the following query: `service:agent` you are able to see the parsed agent log even if they won't be indexed:

![live_tail_agent](images/logs_workshop/live_tail_agent.png)

# Multi-line logs

Let's kill XXXX and see what happen: 


We need to specify to the Agent to wrap logs according to a begining pattern


Let's kill it again:


### Best practice

If you intend to analyse your logs in a log management tool it's a best practice to use JSON formated logs. Beyond handeling Stack Traces it's one of the only format that allows Human and machine comprehension and manipulation of logs.



# Parsing a full text log into JSON

Most of the time all your logs won't be in JSON format, and if they are their attributes my differ between two log sources


Let's take the following log emited by:


And let's transform it to extract: 


We are going to follow the Datadog Attribute Naming convention: 


### Bonus

To implement even more the Attribute Naming convention we can remap the stacktrace to `error.stack`

# Monitoring your logs

Let's build a monitor upon our logs that warns us if an stack trace occures and that send us our stack trace


### Monitor notification

